In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
import time

df = []
table_all=[]
# scarbing the latest 30 years
for year in range(1988,2019):
    driver=webdriver.Chrome()                
    driver.get('https://www.basketball-reference.com/playoffs/NBA_' + str(year) + '.html')       
    time.sleep(1)                            
    def execute_times(times):
        for i in range(times + 1):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
    execute_times(1)
    html=driver.page_source
    driver.close()
    soup=BeautifulSoup(html,'lxml')
        
    #Team Per Game Stats
    head_off = ["Team"]
    for i in range(3, 25):
        head_off.append(soup.find_all("div", {"class": "table_outer_container"})[1].find_all("th")[i].text)
    table = []
    for i in range(0,408):
        table.append(soup.find_all("div", {"class": "table_outer_container"})[1].find_all("td")[i].text)
    table_off = np.array(table).reshape([17,24])
    #delte the games and minutes played
    table_off = np.delete(table_off,[1,2], axis =1)
 
    #Team Shooting
    if year<=2000: # before year 2000, there are no data set about team shooting, so we should consider in other situation
        table_shooting = np.repeat(np.nan,408).reshape(17,24)
        table = []
        for i in range(0,340):
            table.append(soup.find_all("div", {"class": "overthrow table_container"})[5].find_all("td")[i].text)
        table_adv = np.array(table).reshape([17,20])
        table_adv = np.delete(table_adv, [0,1,2,3,4], axis = 1)
    else:
        table = []
        for i in range(17,476):
            table.append(soup.find_all("div", {"class": "table_outer_container mobile_table"})[0].find_all("td")[i].text)
        table_shooting = np.array(table).reshape([17,27])
        table_shooting = np.delete(table_shooting, [0,1,2], axis = 1)
        head_shotting= ['FG%', 'Dist.', '2P(FGA)', '0-3(FGA)', '3-10(FGA)', '10-16(FGA)', '16 <3(FGA)', '3P(FGA)', '2P(FG)', '0-3(FG)', '3-10(FG)', '10-16(FG)', 
                        '16 <3(FG)', '3P(FG)', "%Ast'd(2PT)", '%FGA(2PTDunk)', 'Md.(2PTDunk)', '%FGA(2PTLayups)','Md.(2PTLayups)', "%Ast'd(3PT)",
                        '%3PA(3PTLayout)', '3P%((3PT)Layout)','Att.','Md.'] 
        # Miscellaneous Stats
        head = []
        for i in range(10,25):
            head.append(soup.find_all("div", {"class": "overthrow table_container"})[7].find_all("th")[i].text)
        table = []
        head_adv = ['PW','PL','ORtg','DRtg','Pace','FTr','3PAr','eFG%(offens)','TOV%(offens)','ORB%(offens)','FT/FGA(offens)','eFG%(defense)','TOV%(defense)',
        'DRB%(defense)','FT/FGA(defense)','Year']
        for i in range(0,340):
            table.append(soup.find_all("div", {"class": "overthrow table_container"})[7].find_all("td")[i].text)
        table_adv = np.array(table).reshape([17,20])
        table_adv = np.delete(table_adv, [0,1,2,3,4], axis = 1)
    table = np.append(table_off,np.append(table_shooting, table_adv, axis=1), axis = 1)
    y = np.ones(table.shape[0])*year
    table = np.c_[table,y]
    if len(table_all)==0:
        table_all = table
    else:
        table_all = np.append(table_all, table, axis = 0)
df = pd.DataFrame(table_all)
df.columns = head_off + head_shotting + head_adv
df.to_csv("/Users/apple/Desktop/df.csv")